# 01 – EDA y Transformaciones
**Prueba técnica – Ingeniero de Datos**

Este notebook explora el dataset *dataset_salud_500k.csv*, identifica problemas de calidad y aplica transformaciones básicas para producir un archivo *clean_dataset.parquet* listo para carga.

*Generado automáticamente: 2025-05-16 19:08:58*

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from src.config import RAW_CSV_PATH, CLEAN_PARQUET_PATH, CLEAN_DATA_DIR
from scipy import stats

plt.style.use("default")
pd.set_option("display.max_columns", None)

SAMPLE_FRAC = 0.20
RANDOM_STATE = 42

In [ ]:

%%time
df_full = pd.read_csv(RAW_CSV_PATH, low_memory=False)
df = df_full.sample(frac=SAMPLE_FRAC, random_state=RANDOM_STATE).reset_index(drop=True)

print(f"Shape completo: {df_full.shape} | Muestra: {df.shape}")
display(df.head())
display(df.info(memory_usage='deep'))



## Principales hallazgos del EDA rápido

| Métrica | Valor aproximado |
|---------|------------------|
| Filas totales | 500 000 |
| Columnas | 16 |
| Nulos — `gender` | ~4 % |
| Nulos — `visit_type` | ~1 % |
| Edades negativas o >120 | ~6 % |
| Costos = 0 USD | ~1 % |
| Costos > 50 000 USD | <1 % |
| Especialidades con typos | `Cardiolgia`, `Ginecplogía`, `Cirujia`, `Pediatra` |
| Duplicados exactos | 0 % |


In [ ]:

# Resumen numérico
num_cols = df.select_dtypes(include='number').columns
display(df[num_cols].describe().T)

# Resumen categórico
cat_cols = df.select_dtypes(exclude='number').columns
display(df[cat_cols].describe().T)


In [ ]:

missing = (df.isna().mean()*100).round(2).sort_values(ascending=False)
display(missing.to_frame('pct_missing'))


In [ ]:

fig, axes = plt.subplots(1, 2, figsize=(12,4))
df['age'].dropna().hist(bins=40, ax=axes[0])
axes[0].set_title('Distribución de edad')

df['cost_usd'].dropna().hist(bins=60, ax=axes[1])
axes[1].set_title('Distribución de costos (USD)')
plt.tight_layout()



## Transformaciones y limpieza

A continuación se aplican reglas mínimas para preparar **`df_clean`**:

1. Columnas a `snake_case`.<br>
2. Conversión `visit_date` a fecha válida.<br>
3. Filtrado de edades (0–120).<br>
4. Eliminación de costos negativos, imputación de nulos con la mediana.<br>
5. `gender` faltante → `'unknown'`.<br>
6. Normalización de especialidades con mapa de typos.<br>
7. Guardado a Parquet.


In [ ]:
from src.config import CLEAN_DATA_DIR, CLEAN_PARQUET_PATH
CLEAN_DATA_DIR.mkdir(parents=True, exist_ok=True)

df_clean = df_full.copy()

# 1. snake_case columnas
df_clean.columns = (
    df_clean.columns
            .str.strip()
            .str.lower()
            .str.replace(r'[^a-z0-9]+', '_', regex=True)
            .str.strip('_')
)

# 2. visit_date
df_clean['visit_date'] = pd.to_datetime(df_clean['visit_date'], errors='coerce')
df_clean = df_clean.dropna(subset=['visit_date'])

# 3. edades válidas
df_clean['age'] = pd.to_numeric(df_clean['age'], errors='coerce')
df_clean = df_clean[df_clean['age'].between(0, 120, inclusive='both')]

# 4. costos
df_clean['cost_usd'] = pd.to_numeric(df_clean['cost_usd'], errors='coerce')
df_clean = df_clean[df_clean['cost_usd'] >= 0]
median_cost = df_clean['cost_usd'].median()
df_clean['cost_usd'] = df_clean['cost_usd'].fillna(median_cost)

# 5. gender
df_clean['gender'] = df_clean['gender'].fillna('unknown')

# 6. normalización especialidades
typo_map = {
    'Cardiolgia': 'Cardiología',
    'Ginecplogía': 'Ginecología',
    'Cirujia': 'Cirugía',
    'Pediatra': 'Pediatría'
}
df_clean['specialty'] = df_clean['specialty'].replace(typo_map)

# 7. Guardar
out_path = CLEAN_PARQUET_PATH  # from config Path('clean_dataset.parquet')
df_clean.to_parquet(out_path, index=False)
print(f'Parquet guardado en {out_path.resolve()} – filas resultantes: {len(df_clean):,}')
